In [1]:
import numpy as np
import copy
import spires

In [265]:
# Example usage
def objective(x):
    return (x[0] - 0.5) ** 2 + (x[1] - 0.5)**2 + (x[2]-60)** 2 + (x[3] -300)**2 + 3

def constraint(x):
    """Constraint function."""
    return x[0] + x[1] - 1

In [770]:
lut_file = '../tests/data/lut_sentinel2b_b2to12_3um_dust.mat'
interpolator = spires.spires.LutInterpolator(lut_file=lut_file) 
spectrum_target = np.array([0.3424, 0.366, 0.3624, 0.38932347, 0.41624767, 0.39567757, 0.07043362, 0.06267947, 0.3792])
spectrum_background = np.array([0.0182, 0.0265, 0.0283, 0.05606749, 0.09543234, 0.12036866, 0.12491679, 0.07888655, 0.1406])
spectrum_shade = np.zeros_like(spectrum_target)
solar_angle = 55.73733298

def objective(x):
    ret = spires.core.spectrum_difference_scaled(spectrum_background=spectrum_background, 
                                          spectrum_target=spectrum_target,
                                          spectrum_shade=spectrum_shade,
                                          solar_angle=solar_angle,                   
                                          bands=interpolator.bands,
                                          solar_angles=interpolator.solar_angles,
                                          dust_concentrations=interpolator.dust_concentrations,
                                          grain_sizes=interpolator.grain_sizes,        
                                          lut=interpolator.reflectances,
                                          x=x)

    return ret

In [771]:
ret, spect = spires.invert.speedy_invert_scipy(interpolator=interpolator, 
                                              spectrum_background=spectrum_background, 
                                  spectrum_target=spectrum_target, 
                                  solar_angle=solar_angle, mode=4)

In [772]:
ret

In [427]:
res = spires.core.invert_(spectrum_background=spectrum_background, 
           spectrum_target=spectrum_target,
           spectrum_shade=spectrum_shade,                          
           solar_angle=solar_angle, 
            bands=interpolator.bands,
                    solar_angles=interpolator.solar_angles,
                                          dust_concentrations=interpolator.dust_concentrations,
                                          grain_sizes=interpolator.grain_sizes,        
                                          lut=interpolator.reflectances,                         
           max_eval=25)

In [428]:
res

In [442]:
bounds = np.array([[0.0, 1.0], 
                   [0.0, 1.0],
                   [0.0, 1.0],
                   [0.0, 1.0]])  # Bounds for each variable

In [481]:
xnew

In [768]:
"""COBYLA optimization for a 2D function."""
x = copy.copy(np.array(x0))
xold = copy.copy(x0)
rhobeg = 0.05
rhoend = 1e-3
maxiter = 1000
x = np.array([0.5, 0.05, 0.01, 0.2])

for _ in range(maxiter):        
    gradient = [0, 0, 0, 0]

    ## Finding the gradient in each dimension by wiggling along rho
    for i in range(len(x)):
        dx = [0, 0, 0, 0]
        dx[i] = rhobeg
        gradient[i] = (objective(x + dx) - objective(x) ) / rho 
    
    if constraint(x) >= 0:
        linear_constraint = np.zeros_like(x)
        for i in range(0):
            dx = np.zeros_like(x)
            dx[i] = rho
            linear_constraint[i] = (constraint(x + dx) - constraint(x)) / rho
        if np.any(linear_constraint < 0):
            gradient = np.maximum(gradient, -linear_constraint)
    
    x = x - np.array(gradient)# * constraint(x) #/ np.linalg.norm(gradient)     
    x = np.clip(x, bounds[:, 0], bounds[:, 1])

    print(np.linalg.norm(objective(x) - objective(xold)))
    if np.linalg.norm(objective(x) - objective(xold)) < rhoend:        
        break

    xold = copy.copy(x)
    
    rhobeg *= 0.5  # Reduce the trust region size in each iteration
print(_)

In [769]:
print(x)
print(x[0], x[1])
print(spires.index_to_value(x[2], interpolator.dust_concentrations))
print(interpolator.grain_sizes.max() * x[3])
objective(x)

1. evaluate how far are the old guesses from the new guesses (sum_)
2. evaluate how fare the old and new objective is
3. calculate the step: The further 